In [2]:
#feature extraction
import glob
import librosa 
import numpy as np
import pickle

def createSentenceAsMelSpectrogram(filename):
    #convert all wav files into mel spectrogram features
    y,sr = librosa.load(filename)
    inputfeatureVectorMatrix=librosa.feature.melspectrogram(y=y,sr=sr,power=1) #power=1 is energys
    #keep ground truth labels for all vectors - can get this from filenames for soundscapes - should do a dictionary tbh
    #convert list of lists to 2Dnumpy array 
 #   librosa.display.specshow(inputfeatureVectorMatrix,y_axis='mel', x_axis='time')
    featVecMat=np.array(inputfeatureVectorMatrix)
    print(featVecMat.dtype, featVecMat.shape, np.sum(featVecMat))
    #log of melspec
    featVecMat=np.log(featVecMat) 
    print(featVecMat.dtype, featVecMat.shape, np.sum(featVecMat))
 #   librosa.display.specshow(featVecMat,y_axis='mel', x_axis='time')
    savefilename = filename.split('.')[0]+'_logMelSpec.pckl'
    savefilename = savefilename.replace('audio','logMelSpec')
    print(savefilename)
    with open(savefilename, 'wb') as sf:
        pickle.dump((featVecMat), sf, protocol=pickle.HIGHEST_PROTOCOL)

samplelist=[]
#this creates scene files - will slice for events afterwards
#src_soundscenes='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\audio\\*.wav'
#audiofilelist=glob.glob(src_soundscenes)
#for afile in audiofilelist:
#    print(afile)
   # samplelist.append(afile.split('.')[0]+'_logMelSpec.pckl'.replace('audio','logMelSpec'))
   # createSentenceAsMelSpectrogram(afile)
   

In [15]:
# Get long term statistics of scene information 
#option 1 - aggregate & interpolate
import pdb
def eachFrameMean(fv,af):
    fv=np.transpose(fv)
    r,c=fv.shape
    aggScene=np.zeros((r,c))
    for i in range(0,r):
        if i+af > r:
            aggScene[i,:]=np.mean(fv[i:r,:],axis=0)
        else:
            aggScene[i,:]=np.mean(fv[i:i+af,:],axis=0)  
    return np.transpose(aggScene)

src_soundscenes='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\logMelSpec\\*_logMelSpec.pckl'
audiofilelist=glob.glob(src_soundscenes)
for scene in audiofilelist:
    with open(scene, "rb" ) as scenesample:
        (featVecMat) = pickle.load(scenesample)
     #   librosa.display.specshow(featVecMat,y_axis='mel', x_axis='time')
        aggregated_scene = eachFrameMean(featVecMat,25)
     #   librosa.display.specshow(aggregated_scene,y_axis='mel', x_axis='time')
        savefilename = scene.split('_log')[0]+'_aggScenes.pckl'
    with open(savefilename, 'wb') as sf:
        pickle.dump((aggregated_scene), sf, protocol=pickle.HIGHEST_PROTOCOL)


In [4]:
#need to normlise the features before performing dimension reduction/transformations. 
#YOGI - MEANS PER TRAINING/VALIDATION FOLD. CUT out test files first
#to match dcase scenes 13 to keep source scenes separate; 
#test files per fold are F1=S1&S6, F2=S2&S7, F3=S3&S8, F4=S4&S9, F5=S5&S10
import string, pickle,glob
import numpy as np

def fileLists(filelist):
    trainlist=[]
    validationlist=[]
    testlist=[]
    
    evalSetupFiles='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\evaluation_setup\\*.txt'
    txtfilelist=glob.glob(evalSetupFiles)
    for txt in txtfilelist:
        if '_location' not in txt:
            continue
        with open(txt,'r') as evaltxtfile:
            for line in evaltxtfile.readlines():
                line=line.strip().split('\t')[0]
                if 'train' in txt:
                    trainlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                elif 'test' in txt:
                    testlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                else:
                    validationlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
    print('trainfiles: ', str(len(trainlist)))
    print('validationfiles: ', str(len(validationlist)))
    print('testfiles: ', str(len(testlist)))
    return trainlist,validationlist,testlist

def processfold(audiofilelist,folder,pth):
    if folder == 'train':
        first=True
        N=len(audiofilelist)*431
        for scene in audiofilelist:
            with open(pth+scene, "rb" ) as scenesample:
                (featVecMat) = pickle.load(scenesample)
                if first:
                    first=False
                    sumssofar=featVecMat
                    sqsofar=np.square(featVecMat)
                else:
                    sumssofar=np.add(sumssofar,featVecMat)
                    sqsofar=np.add(np.square(featVecMat),featVecMat)
        sqsums=np.zeros(128)
        sqmeans=np.zeros(128)
        for i,e in enumerate(sqsofar):
            sqsums[i]=np.sum(e[i])
            sqmeans[i]=np.sum(sumssofar[i,:])
        freq_means=sqmeans/N
        freq_stds=np.zeros((128))
        freq_stds=np.sqrt(abs((sqsums/N)-np.square(freq_means)))
        print(freq_stds.shape)
        savefoldstats='mean_and_sdev.pckl'
        with open(savefoldstats, 'wb') as sf:
            pickle.dump((freq_means,freq_stds), sf, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        openfoldstats='mean_and_sdev.pckl'
        with open(openfoldstats, "rb" ) as stats:
            (freq_means,freq_stds) = pickle.load(stats)
    print('Normalizing over time')
    freq_means=np.transpose(np.tile(freq_means,(431,1)))
    freq_stds=np.transpose(np.tile(freq_stds,(431,1)))
    print('below shoul be 128,431')
    print(freq_means.shape)
    for scene in audiofilelist:
        with open(pth+scene, "rb" ) as scenesample:
            (featVecMat) = pickle.load(scenesample)
          #  librosa.display.specshow(featVecMat,y_axis='mel', x_axis='time')
            featVecMat=featVecMat-freq_means
            featVecMat=featVecMat/freq_stds
         #   librosa.display.specshow(featVecMat,y_axis='mel', x_axis='time')
        savefilename = pth+scene.replace('logMelSpec',folder)  #change folder
        print(savefilename)
        with open(savefilename, 'wb') as sf:
            pickle.dump((featVecMat), sf, protocol=pickle.HIGHEST_PROTOCOL)
    print('NEXT!')

#normalise scenes by fold
#src_files='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\logMelSpecPckl\\*_aggScenes.pckl'    
#audiofilelist=glob.glob(src_files)
#trainlist,validationlist,testlist=fileLists(audiofilelist)
#print('yo')
#pth='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\'
##processfold(trainlist,'train',pth)
#processfold(validationlist,'val',pth)
#processfold(testlist,'test',pth)  

#print('ta da!')

In [13]:
## extra code to work out the mean and std of features for each city
def calcFeatureMeanAndStd(audiofilelist,classname):
    pth='..\\..\\CASAdatasets\DCASE18_ASCT1\TUT-urban-acoustic-scenes-2018-development\\'
    N=len(audiofilelist)*431
    first=True
    for scene in audiofilelist:
        with open(pth+scene, "rb" ) as scenesample:
            (featVecMat) = pickle.load(scenesample)
            if first:
                first=False
                sumssofar=featVecMat
                sqsofar=np.square(featVecMat)
            else:
                sumssofar=np.add(sumssofar,featVecMat)
                sqsofar=np.add(np.square(featVecMat),featVecMat)
    sqsums=np.zeros(128)
    sqmeans=np.zeros(128)
    for i,e in enumerate(sqsofar):
        sqsums[i]=np.sum(e[i])
        sqmeans[i]=np.sum(sumssofar[i,:])
    freq_means=sqmeans/N
    freq_stds=np.zeros((128))
    freq_stds=np.sqrt(abs((sqsums/N)-np.square(freq_means)))
    print(freq_stds.shape)
    savefoldstats=classname+'_mean_and_sdev.pckl'
    with open(savefoldstats, 'wb') as sf:
        pickle.dump((freq_means,freq_stds), sf, protocol=pickle.HIGHEST_PROTOCOL)
    return (freq_means,freq_stds)

src_files='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\logMelSpecPckl\\*_aggScenes.pckl'    
audiofilelist=glob.glob(src_files)
trainlist,validationlist,testlist=fileLists(audiofilelist)
cities=['barcelona','helsinki','paris','london','stockholm','vienna']
meanstd={}
for city in cities:
    tmptrainlist=[]
    for recording in trainlist:
        if city in recording:
            tmptrainlist.append(recording)
    meanstd[city]=calcFeatureMeanAndStd(tmptrainlist,city)
    



trainfiles:  6122
validationfiles:  2518
testfiles:  2518
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)


In [20]:
## then we work out the euclidean distance between these
import math
def euclidean_distance(x,y):
    return math.sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))

distances={}
for city,(avg,std) in meanstd.items():
    for cty,(av,st) in meanstd.items():
        if city==cty:
            continue
        dis=euclidean_distance(avg,av)
        distances[(city,cty)]=dis
        
for city_pair,dist in distances.items():
    print(city_pair,dist)

('barcelona', 'helsinki') 3.1958103115583816
('barcelona', 'paris') 3.603709950882723
('barcelona', 'london') 3.3085097792172475
('barcelona', 'stockholm') 2.0379798491606143
('barcelona', 'vienna') 4.9312973770878195
('helsinki', 'barcelona') 3.1958103115583816
('helsinki', 'paris') 1.8796792590375553
('helsinki', 'london') 1.353578472237539
('helsinki', 'stockholm') 4.339801492171317
('helsinki', 'vienna') 2.0770472345510256
('paris', 'barcelona') 3.603709950882723
('paris', 'helsinki') 1.8796792590375553
('paris', 'london') 0.6714508764461671
('paris', 'stockholm') 4.77832115733459
('paris', 'vienna') 2.3624188336428356
('london', 'barcelona') 3.3085097792172475
('london', 'helsinki') 1.353578472237539
('london', 'paris') 0.6714508764461671
('london', 'stockholm') 4.481470206487289
('london', 'vienna') 2.1596818377416467
('stockholm', 'barcelona') 2.0379798491606143
('stockholm', 'helsinki') 4.339801492171317
('stockholm', 'paris') 4.77832115733459
('stockholm', 'london') 4.48147020